### 隨機森林進行選股回測 - 以財報資料處理

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import datetime
from pandas.tseries.offsets import BMonthEnd
from pandas.tseries.offsets import MonthEnd

#### Step1.先讀取每日報酬率(已轉換成累積一季的結果)

In [2]:
ret_date = pd.read_pickle('每季報酬(日資料表示).pkl')
ret_date

,證券代碼,年月日,累積季報酬率
0,1101 台泥,20120924,0.084860
1,1101 台泥,20120925,0.083558
2,1101 台泥,20120926,0.046240
3,1101 台泥,20120927,0.052090
4,1101 台泥,20120928,0.068239
...,...,...,...
4457108,9962 有益,20240624,0.006183
4457109,9962 有益,20240625,-0.007949
4457110,9962 有益,20240626,-0.013707
4457111,9962 有益,20240627,-0.022165


> 記得轉換日期，把ret_date整理成方便閱讀

In [3]:
ret_date['年月日'] = pd.to_datetime(ret_date['年月日'],format='%Y%m%d')
ret_date

,證券代碼,年月日,累積季報酬率
0,1101 台泥,2012-09-24,0.084860
1,1101 台泥,2012-09-25,0.083558
2,1101 台泥,2012-09-26,0.046240
3,1101 台泥,2012-09-27,0.052090
4,1101 台泥,2012-09-28,0.068239
...,...,...,...
4457108,9962 有益,2024-06-24,0.006183
4457109,9962 有益,2024-06-25,-0.007949
4457110,9962 有益,2024-06-26,-0.013707
4457111,9962 有益,2024-06-27,-0.022165


#### Step2.讀取財報資訊(Tej所有的財報資料都下載並合併)

In [4]:
fin_date = pd.read_table('財報資料.txt',encoding = 'ANSI')
fin_date

,證券代碼,年月,財報發布日,現金及約當現金,透過損益按公允價值衡量之金融資產－流動,透過其他綜合損益按公允價值衡量之金融資產－流動,按攤銷後成本衡量之金融資產－流動,避險之金融資產－流動,合約資產－流動,應收帳款及票據,...,每人營業利益,每人配備率,季底普通股市值,當季季底P/E,當季季底P/B,當季季底PSR,股利殖利率,現金股利率,Tobins Q,Tobins Q (A)
0,000116 日盛證券,201303,2013/05/15,6389095,11450720,0.0,0,0.0,NaN,4558590.0,...,79.0,862.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000930 華南永昌證券,201303,2013/05/13,1159077,4516758,4996889.0,0,0.0,NaN,4218493.0,...,17.0,1015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000960 富邦證券,201303,2013/05/14,6956614,4810810,6273480.0,0,0.0,NaN,6011923.0,...,43.0,697.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0009A0 永豐金證券,201303,2013/05/15,3183136,26042573,0.0,0,0.0,NaN,6388999.0,...,69.0,976.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1101 台泥,201303,2013/05/15,18994984,271687,15066125.0,0,6836.0,0.0,18645075.0,...,207.0,11344.0,138456563.0,12.60,1.28,1.23,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78267,8271 宇瞻,202409,2024/10/28,1180838,79438,0.0,0,0.0,0.0,845366.0,...,63.0,1542.0,6732527.0,12.95,1.51,NaN,NaN,NaN,NaN,NaN
78268,8279 生展,202409,2024/10/31,228310,0,0.0,0,0.0,0.0,477197.0,...,NaN,NaN,3590883.0,12.03,1.65,NaN,NaN,NaN,NaN,NaN
78269,8374 羅昇,202409,2024/11/01,996068,781,0.0,1016,0.0,12070.0,935456.0,...,133.0,1531.0,15658875.0,301.98,7.89,NaN,NaN,NaN,NaN,NaN
78270,8454 富邦媒,202409,2024/11/01,3332445,0,142.0,0,0.0,0.0,459073.0,...,233.0,2482.0,99428658.0,22.95,11.13,NaN,NaN,NaN,NaN,NaN


> 依照"證券代碼"、"年月"進行排序比較好看

In [5]:
fin_date = fin_date.sort_values(by=["證券代碼", "年月"], ascending=[True, True])
fin_date

,證券代碼,年月,財報發布日,現金及約當現金,透過損益按公允價值衡量之金融資產－流動,透過其他綜合損益按公允價值衡量之金融資產－流動,按攤銷後成本衡量之金融資產－流動,避險之金融資產－流動,合約資產－流動,應收帳款及票據,...,每人營業利益,每人配備率,季底普通股市值,當季季底P/E,當季季底P/B,當季季底PSR,股利殖利率,現金股利率,Tobins Q,Tobins Q (A)
0,000116 日盛證券,201303,2013/05/15,6389095,11450720,0.0,0,0.0,NaN,4558590.0,...,79.0,862.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1506,000116 日盛證券,201306,2013/08/29,5025612,9900538,0.0,0,0.0,NaN,4920951.0,...,56.0,852.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3169,000116 日盛證券,201309,2013/11/13,5506843,7606085,0.0,0,0.0,NaN,4601471.0,...,58.0,844.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4690,000116 日盛證券,201312,2014/03/27,4952504,8223016,0.0,0,0.0,NaN,6670711.0,...,56.0,844.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6347,000116 日盛證券,201403,2014/05/14,4101975,10950632,0.0,0,0.0,NaN,6557725.0,...,151.0,835.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71103,9962 有益,202306,2023/08/09,150008,0,0.0,0,0.0,0.0,100317.0,...,373.0,3035.0,1840488.0,7.88,1.65,NaN,NaN,NaN,NaN,NaN
72853,9962 有益,202309,2023/11/13,177119,0,0.0,0,0.0,0.0,76077.0,...,327.0,3384.0,1538251.0,7.95,1.35,0.43,NaN,NaN,NaN,NaN
74624,9962 有益,202312,2024/03/12,173667,0,0.0,0,0.0,0.0,57948.0,...,225.0,3510.0,1646515.0,11.54,1.43,NaN,NaN,NaN,NaN,NaN
76382,9962 有益,202403,2024/05/08,162474,0,0.0,0,0.0,0.0,48363.0,...,NaN,NaN,1619449.0,16.66,1.53,NaN,NaN,NaN,NaN,NaN


In [6]:
## 將財報發布日轉成月底的那一天，並轉成該月月底最後一個"工作天"，這樣每月都可以統一決定是否換股
fin_date["財報發布日"] = pd.to_datetime(fin_date["財報發布日"], errors='coerce')
fin_date["財報發布日"] = fin_date["財報發布日"] + BMonthEnd(0)

## 更改column名稱，方便合併DataFrame
fin_date.rename(columns={"公司": "證券代碼","財報發布日": "年月日","年月": "季"}, inplace=True)

In [7]:
fin_date

,證券代碼,季,年月日,現金及約當現金,透過損益按公允價值衡量之金融資產－流動,透過其他綜合損益按公允價值衡量之金融資產－流動,按攤銷後成本衡量之金融資產－流動,避險之金融資產－流動,合約資產－流動,應收帳款及票據,...,每人營業利益,每人配備率,季底普通股市值,當季季底P/E,當季季底P/B,當季季底PSR,股利殖利率,現金股利率,Tobins Q,Tobins Q (A)
0,000116 日盛證券,201303,2013-05-31,6389095,11450720,0.0,0,0.0,NaN,4558590.0,...,79.0,862.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1506,000116 日盛證券,201306,2013-08-30,5025612,9900538,0.0,0,0.0,NaN,4920951.0,...,56.0,852.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3169,000116 日盛證券,201309,2013-11-29,5506843,7606085,0.0,0,0.0,NaN,4601471.0,...,58.0,844.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4690,000116 日盛證券,201312,2014-03-31,4952504,8223016,0.0,0,0.0,NaN,6670711.0,...,56.0,844.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6347,000116 日盛證券,201403,2014-05-30,4101975,10950632,0.0,0,0.0,NaN,6557725.0,...,151.0,835.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71103,9962 有益,202306,2023-08-31,150008,0,0.0,0,0.0,0.0,100317.0,...,373.0,3035.0,1840488.0,7.88,1.65,NaN,NaN,NaN,NaN,NaN
72853,9962 有益,202309,2023-11-30,177119,0,0.0,0,0.0,0.0,76077.0,...,327.0,3384.0,1538251.0,7.95,1.35,0.43,NaN,NaN,NaN,NaN
74624,9962 有益,202312,2024-03-29,173667,0,0.0,0,0.0,0.0,57948.0,...,225.0,3510.0,1646515.0,11.54,1.43,NaN,NaN,NaN,NaN,NaN
76382,9962 有益,202403,2024-05-31,162474,0,0.0,0,0.0,0.0,48363.0,...,NaN,NaN,1619449.0,16.66,1.53,NaN,NaN,NaN,NaN,NaN


In [ ]:
## 新增fin_date一個欄位，是3個月過後的月底那天，這樣在合併的時候才可以合併到買進一季的報酬率

fin_date["合併日期"] = pd.to_datetime(fin_date["年月日"]) + BMonthEnd(3)
ret_date = ret_date.set_index(["證券代碼", "年月日"])
fin_date = fin_date.set_index(["證券代碼", "合併日期"])

In [ ]:
ret_date

In [ ]:
fin_date

#### Step3.合併兩邊的DataFrame

In [ ]:
merge = fin_date.merge(ret_date,  left_index=True, right_index=True)
#merge = merge.drop(columns=['當月月底'])
merge

#### Step4.將所有Column都轉換為數值(這邊Tej下載下來的格式有時候可能會有問題)

In [ ]:
merge = merge.apply(pd.to_numeric, errors='coerce')

In [ ]:
#切割訓練 測試集, 將缺失值補0
condition = merge.index.get_level_values('對照用日期') < '2020-06-30'
train_data = merge[condition].fillna(0)
test_data = merge[~condition].fillna(0)

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
#跑模型
rf = RandomForestClassifier(n_estimators=1000,max_depth=30,criterion= 'entropy')
rf.fit(train_data.drop(columns = '累積季報酬率'), train_data['累積季報酬率'] > 0)

In [ ]:
#訓練集、測試集分數
print("訓練集分數: " , rf.score(train_data.drop(columns = '累積季報酬率'), train_data['累積季報酬率'] > 0))
print("測試集分數: " , rf.score(test_data.drop(columns = '累積季報酬率'), test_data['累積季報酬率'] > 0))

In [ ]:
#回測
selected = rf.predict(test_data.drop(columns = '季報酬率'))

In [ ]:
test_data[selected]

In [ ]:
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']  #顯示中文
(test_data[selected].groupby('日期').mean()['季報酬率'] + 1).cumprod().plot(color = 'blue')     #randomforest
(test_data[~selected].groupby('日期').mean()['季報酬率'] + 1).cumprod().plot(color = 'orange')  #benchmark1
(test_data.groupby('日期').mean()['季報酬率'] + 1).cumprod().plot(color = 'red')                #benchmark2

In [ ]:
#看權重, 看那些特徵比較重要
feature_name = train_data.columns[:-1]
important = pd.Series(rf.feature_importances_, index = feature_name).sort_values(ascending=False)
important.head(20)